In [1]:
import os


In [2]:
%pwd

'D:\\projects\\chicken-disease-classification-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'D:\\projects\\chicken-disease-classification-project'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file: Path
    unzip_dir: Path

In [17]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            params_filepath= PARAMS_FILE_PATH ):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
             root_dir=config.root_dir,
             source_URL=config.local_data_file,
             local_data_file=config.local_data_file,
             unzip_dir=config.unzip_dir
        )

        return data_ingestion_config



In [32]:
import os
import zipfile
import urllib.request as request
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [ ]:
import os
import zipfile
import urllib.request as request
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Helper function to get file size
def get_size(path: Path) -> str:
    size_in_bytes = path.stat().st_size
    size_in_mb = size_in_bytes / (1024 * 1024)
    return f"{size_in_mb:.2f} MB"

# DataIngestionConfig Class
class DataIngestionConfig:
    def __init__(self, source_URL: str, local_data_file: str, unzip_dir: str):
        self.source_URL = source_URL
        self.local_data_file = local_data_file
        self.unzip_dir = unzip_dir

# DataIngestion Class
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! With the following info: \n{headers}")
        else:
            logger.info(
                f"File already exists. Size: {get_size(Path(self.config.local_data_file))}"
            )

    def extract_zip_file(self):
        """
        Extracts the zip file into the specified directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted all files to {unzip_path}")


In [51]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-21 11:27:56,637: INFO: 29816721: Looking for config file at: d:\projects\chicken-disease-classification-project\research\..\config\config.yaml]
[2024-11-21 11:27:56,644: INFO: common: yaml file: d:\projects\chicken-disease-classification-project\research\..\config\config.yaml loaded successfully]
[2024-11-21 11:27:56,650: INFO: common: created directory at: artifacts/data_ingestion]
[2024-11-21 11:27:56,654: INFO: 29816721: File already exists. Size: 11.08 MB]
[2024-11-21 11:27:57,115: INFO: 29816721: Extracted all files to artifacts/data_ingestion]
